In [1]:
# Making a function StockVol to calibrate stock volitility under geometric Brownian motion model
!pip install pandas_datareader
!pip install yfinance
!pip install cvxpy
import numpy as np
import matplotlib as m
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import data
import yfinance as yf
import datetime as dt
from datetime import date
yf.pdr_override()

/home/mayuresh/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
#USE    : To get historical data for a particular stock
#INPUT  : code of the company "goog"
#OUTPUT : array of close price of data
def GetData(companyName, years=1):
    # Getting today's date
    today = date.today()
    # Setting start date to historical years
    startDate = today.replace(today.year-years)
    # Date : Open High Low Close "Adj Close" Volume
    data = pdr.get_data_yahoo(companyName, startDate, today)
    # Cleaning to get the close prices only
    closePrice = [data["Close"][i] for i in range(len(data))]
    # Return close price of the data
    return closePrice

In [3]:
#USE        : to find the volitility of the stock data
#INPUT      : array of 1 year historical prices
#ASSUMPTION : Stock doesn't pay dividends
#OUTPUT     : historical volitility of the stock (standard deviation)
def StockVol(histoPrice):
    #Finding the ratio of current/previous values
    Sn = [np.log(histoPrice[i+1]/histoPrice[i]) for i in range(len(histoPrice)-1)]
    return np.sqrt(np.var(Sn, ddof=1))

In [4]:
#USE        : to generate n stock path
#INPUT      : nSteps-> number of time steps, sigma-> volitility, 
#           : T-> TerminalTime (yearly unit), nSimulations-> number of Simulations
#           : r-> interest rate, delta-> continuous dividend yield 
#           : S0-> initial price of the stock
#ASSUMPTION : Stock is not paying any dividends
#OUTPUT     : Stock Path as a matrix
def StockPath(nSteps, sigma = 0, S0=0, T=1, nSimulations=10, r=.02, delta=0):
    path = []
    for period in range(nSimulations):
        # Step size for every path simulation
        step =  T/nSteps
        #
        periodPrice = [S0]
        # Computation of n simulated stock prices 
        # np.random.normal(0,1,n) outputs array of nSteps normal values
        Y = np.exp(((r-delta-(sigma**2)/2)*step)+(sigma*np.random.normal(0,1,nSteps)*np.sqrt(step)))
        # now we have array of simulated factor for nSteps stock path 
        # we want to multiply this to previous to find simulated price
        counter = 0
        for y in Y:
            periodPrice.append(periodPrice[counter]*y)
            counter += 1
        path.append(periodPrice)
    return(path)      

In [5]:
#USE        : To generate the European put option price through Monte Carlo method
#INPUT      : nSteps-> number of time steps, sigma-> volitility, 
#           : T-> TerminalTime (yearly unit), nSimulations-> number of Simulations
#           : r-> interest rate, delta-> continuous dividend yield 
#           : S0-> initial price of the stock
#ASSUMPTION : Stock is not paying any dividends
#OUTPUTS    : Discounted Payoff Vector, Price & Variance
def EurOptPrice(StockPath, nStockPath, T=1, r=.02, K = 0):
    # Last Column of StockPath is Terminal Value
    Dis_Payoff_Vec = []
    for j in range(0,nStockPath):
        St_j = StockPath[j][nStockPath]
        # Create Column of Disounted Payoffs
        Dis_Payoff_j = np.exp(-r*T)*max(0,K - St_j)
        Dis_Payoff_Vec = np.append(Dis_Payoff_Vec,Dis_Payoff_j)
        Price = np.mean(Dis_Payoff_Vec)
        Price_Var = np.var(Dis_Payoff_Vec)
    return (Dis_Payoff_Vec, Price, Price_Var)

In [6]:
HistoPrice = GetData("goog", years=1)
sigma = StockVol(HistoPrice)
StockPaths = StockPath(nSteps = 20, sigma = sigma, S0 = HistoPrice[0], T = 1, nSimulations = 52, r = 0.02)
Eu = EurOptPrice(StockPaths, nStockPath = 20, T = 1, r = 0.02, K = 1200)

In [31]:
import cvxpy as cp
def AmericanOptionPrice(StockPaths, nSteps, nSimulations, sigma, r, K, T, mcSamples=1000):
    step = T/nSteps
    # Shape is (nSimulations, nSteps)
    # Take a closer look, all stock path has been reversed to work in the backward direction
    # i.e. it starts from time in reverse direction
    paths = np.array(StockPaths).T[::-1] 
    print(f"shape of paths is {np.shape(paths)}")
    
    # This contains payoffs of all the path or 0th time step
    exercisedPayoffs = [(K-paths[0]).clip(min=0)]
    exercisedExpDisPayoffs = [(K-paths[0]).clip(min=0)]
    
    for i in range(1,nSteps):
        # Has number of values equal to nSsimulations i.e. every path value
        # So basically variable 'paths' contain price at each time step for all simulation
        # stockPrices has price at time step i for every path of simulation
        stockPrices = paths[i]
        print(f"stockPrices shape is {np.shape(stockPrices)}")
        
        # Make positive payoff using clip, this is payofsf at time step i for all path 
        payOffs = (K - stockPrices).clip(min=0)
        print(payOffs, np.shape(payOffs))
        
        # We need to find Discounted Conditional Expected Price for next step of each step value
        # So we need to Simulate 1 step Monte Carlo to generate next time stock price
        # Find the payoff of this next time stock price and calculate average 
        # We use blackSholes model for price simulation
        
        # For every path in the stockPrices simulate Monte Carlo and get next prices using Black-Sholes
        # time step is ith
        simulatedPrices = [ stockPrice*np.exp(sigma*np.random.normal(0,1,mcSamples)*np.sqrt(step) + (r-(sigma**2)/2)*step) for stockPrice in stockPrices]
        simulatedPrices = np.array(simulatedPrices)
        print(f"Shape of simulatedPrices is: {np.shape(simulatedPrices)}")
        # shape is nSimulations X mcSamples
        # So for every path at timeStep i we have mcSamples montecarlo samples simulated
        
        #If this is the t-1 step then the expected payoff is to be simulated mean
        if(i == 1):
            # Find expected payoff by taking mean of all the path
            expectedPayoffs = np.array([np.mean(expectedPayoff) for expectedPayoff in K-simulatedPrices.clip(min=0)])
            print(f"Shape of expectedPayoffs is {np.shape(expectedPayoffs)}")
            discountedExpectedPayoffs = expectedPayoffs* np.exp(-r*step)
            print(f"Shape of discountedExpectedPayoff is {np.shape(discountedExpectedPayoffs)}")
            
        # For other time step it must be done by machine learning
        else:
            # discountedExpectedPayoffs needs to be calculated using the machine learning
            # Take as input stockPrices (has stock price for each path at time step i)
            # and take another input as discountedExpectedPayoffs (has expectedPayoffs 
            # for each path at time step i)
            # run the machine learning model on this to get the weights
            # use this weights to predict new expected payoff at timestep i-1
            
            X = np.array([np.array([1]*len(stockPrices)), stockPrices, stockPrices**2, stockPrices**3])
            X = X.T
            
            # Formalizing the regression model
            beta = cp.Variable(4)
            loss = cp.sum_squares(discountedExpectedPayoffs-X@beta)
            prob = cp.Problem(cp.Minimize(loss))
            prob.solve()
            # estimating discounted expected payoffs using estimated parameters
            discountedExpectedPayoffs = X@beta.value
        
        exercisedPayoffs.insert(0, payOffs)
        exercisedExpDisPayoffs.insert(0, discountedExpectedPayoffs)
    
        
    return(exercisedPayoffs, exercisedExpDisPayoffs)

In [39]:
HistoPrice = GetData("goog", years=1)
sigma = StockVol(HistoPrice)
nSteps = 20
S0 = HistoPrice[0]
StockPaths = StockPath(nSteps = 20, sigma = sigma, S0 = HistoPrice[0], T = 1, nSimulations = 52, r = 0.02)
mcSamples = 1000
nSimulations = 52
r = 0.04
K = 1200
T = 1

In [40]:
a,b = AmericanOptionPrice(StockPaths, nSteps, nSimulations, sigma, r, K, T, mcSamples=1000)

shape of paths is (21, 52)
stockPrices shape is (52,)
[149.8462294  108.81931369 145.19304119 167.78638604 117.24911241
 128.55638789 150.24463959 136.46072888 120.44082687 100.98931963
 139.77193292 124.28239679  80.62939422 175.11192496 137.79296114
  89.80613113 134.51887529 116.43080861 141.61936346 148.52526067
  98.57454398 139.84616588 155.29311432 125.65514135 116.76036589
 116.11494166 147.54086357 136.47985827 132.16054867 129.30988117
 133.49281957 112.08853647 129.30994484 120.5448564  107.64233838
 134.62219756  99.64542913 140.58941722 124.61985156 158.39210911
 150.03579421 129.35110389 139.33459119 125.50958251 159.23399534
 134.13523405 143.45154677 139.06533662 159.27125636 109.70718293
 130.45118311 119.22948417] (52,)
Shape of simulatedPrices is: (52, 1000)
Shape of expectedPayoffs is (52,)
Shape of discountedExpectedPayoff is (52,)
stockPrices shape is (52,)
[157.80763475 111.27860195 152.55377606 159.82139131 120.62308841
 132.61710993 150.14320999 135.48946881 12

Shape of simulatedPrices is: (52, 1000)
stockPrices shape is (52,)
[148.51939555 153.09698224 144.88388836 151.11372585 148.2688266
 139.40681824 158.54209223 149.54877622 144.40260555 152.83235059
 142.51035993 139.67417014 135.80203592 153.222214   152.26087577
 150.49242057 159.66994281 150.39963834 132.38193682 151.67747698
 136.74106708 139.05843101 151.39470571 144.95779314 141.92459873
 141.66282302 144.90390954 143.7166395  150.53325647 138.99761935
 145.35223507 149.50229351 164.65241111 151.68630106 140.13329926
 147.57994782 135.59752749 152.59778315 143.85940844 155.82270394
 145.04512507 135.95268287 137.74819823 144.51078459 148.51803662
 141.09109355 132.85932975 146.14476207 152.68015019 141.74194355
 166.21350763 156.51068239] (52,)
Shape of simulatedPrices is: (52, 1000)
stockPrices shape is (52,)
[149.85134431 149.45674818 142.23186929 150.93052574 144.78236624
 147.20702682 155.39377065 149.4750303  148.17471511 152.31367095
 144.46276071 141.08621942 144.49328132 1

In [48]:
print(a[0])
print(b[0])

[149.85134431 149.45674818 142.23186929 150.93052574 144.78236624
 147.20702682 155.39377065 149.4750303  148.17471511 152.31367095
 144.46276071 141.08621942 144.49328132 147.61585448 148.50784122
 147.98300025 152.76532074 149.02106695 134.58902343 155.64091124
 134.31827376 137.25869632 149.31420666 147.17700875 148.86167747
 139.52277076 144.03601463 151.53362797 153.17654041 150.57348442
 149.17599152 148.11164279 153.6455099  153.06134686 145.77413154
 149.7656301  137.13305621 152.23591042 145.90644223 156.51414593
 146.67012205 137.38034919 144.44375394 141.4307152  148.61060796
 148.97276362 143.33511262 148.39206115 146.10950293 147.18354417
 152.03339978 155.62309069]
[130.37247208 129.98705005 123.99853035 131.44846563 125.8420356
 127.88591007 136.14401406 130.00480589 128.76826245 132.86812066
 125.59253692 123.28951014 125.61614686 128.25446673 129.07976197
 128.59074275 133.34000525 129.56692715 120.99269134 136.41120314
 120.96999972 121.54872386 129.84895708 127.85910

In [ ]:
simulatedPrices = [ stockPrice*np.exp(sigma*np.random.normal(0,1,mcSamples)*np.sqrt(step) + (r-(sigma^2)/2)*step) for stockPrice in stockPrices]

In [ ]:
10*np.array([[1,2],[1,2],[1,2]])

In [ ]:
simulatedPrice